In [1]:
# カルテの頭書き作成システム　header of medical chart 2024/4 by ucymtr
# 環境　JupyterLab 4.0.11 macbook air M1, MacOS Ventura 13.6.5 オンプレORCA 5.2.0
#      ORCA serverとclient(mac)がLANに接続されている
# credentials.pyというファイルを作り、そのなかでORCA severのip address,id,pwを定義する。
# ２つのdirectory ./work ./doneをあらかじめ作成
# 和暦packageのinstall
# 各モジュールのインポート
#   その際にid,pw,ip addressをcredentials.pyというファイル内に定義
# プログラムの実行前に残りラベル数、ID numberを入力
# プログラムの内容
# ORCA API経由で患者のID,氏名、生年月日等を抽出
#  　nでラベルシールの残りのラベル数を入力する　cf. n=18
#  　id_list = [11523,45672] cd. []のなかに印刷したいID numberを入力する
# エクセルのファイル(./work/output-IDnumber.xlsx)として保存
# そのファイルを一つにまとめエクセルのファイル(日付時間秒.xlsx)として保存
# まとめたファイルを、ラベル屋さんのテンプレート(template-mrh.alym)に差し込みデータとして流し込み
# ラベルシール(A-one,１８面,#31285)にまとめて印刷する
#　ラベル屋さん10において
#  　ファイルを開く(テンプレート.slymファイルを読み込む）
#    印刷、次への下の読み込みでデータファイル(日付時間秒.xlsx, ex.2024-04-01-114546.xlsx)を読み込む
#    次へ/印刷へ/印刷する

In [8]:
pip install japanera

Note: you may need to restart the kernel to use updated packages.


In [2]:
# credentials.pyというファイルを作り、そのなかでORCA severのip address,id,pwを定義して下さい。
# 中身の例
# id = "aaaaa"
# pw = "bbbbb"
# ip_address = "192.168.???.???"
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
from datetime import date, datetime
from japanera import EraDate, EraDateTime, Era
import datetime
import credentials
username = credentials.id
password = credentials.pw
ip_address = credentials.ip_address
import glob
import openpyxl
import shutil
import os

In [4]:
# 抽出ファイルを保存する./workというdirectoryと、使用済のデータを保存する./doneというdirectoryを作成する
# nに残っているラベルの数を入れる　(例　未使用の場合はn = 18)
n = 2

# id_listに印刷したい患者のIDをいれる。maxは18
id_list = [11523,45672]
count = len(id_list)
m = n - count

for id in id_list:
    url = f'http://{ip_address}:8000/api01rv2/patientgetv2?id={id}&format=json'
    res = requests.get(url, auth=HTTPBasicAuth(username, password))    
    patient_info = res.json()
    df = pd.json_normalize(patient_info)
    id = patient_info['patientinfores']['Patient_Information']['Patient_ID']
    #電話番号が記載されていない場合'-'を入力する
    if 'patientinfores.Patient_Information.Home_Address_Information.PhoneNumber2' not in df.columns:
        df['patientinfores.Patient_Information.Home_Address_Information.PhoneNumber2'] = '-'
    if 'patientinfores.Patient_Information.Home_Address_Information.PhoneNumber1' not in df.columns:
        df['patientinfores.Patient_Information.Home_Address_Information.PhoneNumber1'] = '-'
    # オンライン資格確認から住所が転記された場合、WholeAddess2が空白の場合がある。空白の時、全角スペースを入力する。
    if 'patientinfores.Patient_Information.Home_Address_Information.WholeAddress2' not in df.columns:
        df['patientinfores.Patient_Information.Home_Address_Information.WholeAddress2'] = '　'    
    
    #patientinfores.Patient_Information.Sexの0,1をそれぞれ、男、女に変換
    replacement_dict = {'1': '男', '2': '女'}
    df['patientinfores.Patient_Information.Sex'].replace(replacement_dict, inplace=True)
    #誕生日を西暦（和暦）年月とする
    bd = df['patientinfores.Patient_Information.BirthDate'].to_string(index=False)
    dt = datetime.datetime.strptime(bd, '%Y-%m-%d')
    year = dt.year
    month = dt.month
    day = dt.day
    era_date = EraDate(year, month, day)
    bd = era_date.strftime("%Y年(%-K%-Y年)%m月%d日")
    df.at[0,'patientinfores.Patient_Information.BirthDate'] = bd
    #郵便番号にハイフンを入れる
    zip =str(df.at[0,'patientinfores.Patient_Information.Home_Address_Information.Address_ZipCode'])
    formatted_zip = zip[:3] + '-' + zip[3:]
    df.at[0,'patientinfores.Patient_Information.Home_Address_Information.Address_ZipCode'] = formatted_zip
    #町名までの住所と番地以下を合体させて、wholeaddress1にいれる
    ad1 = df['patientinfores.Patient_Information.Home_Address_Information.WholeAddress1'].to_string(index=False)
    ad2 = df['patientinfores.Patient_Information.Home_Address_Information.WholeAddress2'].to_string(index=False)
    ad3 = ad1 + ad2
    df.at[0,'patientinfores.Patient_Information.Home_Address_Information.WholeAddress1'] = ad3
    
    df[['patientinfores.Patient_Information.Patient_ID', 'patientinfores.Patient_Information.WholeName_inKana', 'patientinfores.Patient_Information.WholeName',
    'patientinfores.Patient_Information.Sex', 'patientinfores.Patient_Information.BirthDate', 
    'patientinfores.Patient_Information.Home_Address_Information.Address_ZipCode',
       'patientinfores.Patient_Information.Home_Address_Information.WholeAddress1',
        'patientinfores.Patient_Information.Home_Address_Information.PhoneNumber1',
       'patientinfores.Patient_Information.Home_Address_Information.PhoneNumber2']].to_excel('work/output_{}.xlsx'.format(id), index=False, header=True)


current_datetime = datetime.datetime.now()
formatted_datetime = current_datetime.strftime('%Y-%m-%d-%H%M%S')

datafile_list=glob.glob('work/*.xlsx')
datafile_list.sort()
data_files = [pd.read_excel(datafile_list[i],parse_dates=[0]) for i in range(len(datafile_list))]
all_data = pd.concat(data_files,ignore_index = True)
all_data.to_excel(f'{formatted_datetime}.xlsx', index = False, header = False)

workbook = openpyxl.load_workbook(f'{formatted_datetime}.xlsx')
worksheet = workbook.active

# 先頭から印刷するとプリンタに紙が詰まりやすくなるので、残っているラベルの最後から印刷するように先頭に空白を入れる
#　m=残っているラベル数ー印刷するラベル
for _ in range(m):
    # 先頭に空の行を挿入し、A1に全角スペースを設定　m回繰り返す
    worksheet.insert_rows(0)
    worksheet.cell(row=1, column=1).value = '　'

workbook.save(f'{formatted_datetime}.xlsx')

# 移動元のディレクトリ
source_dir = './work'

# 移動先のディレクトリ
destination_dir = './done'

# ファイルを移動
for file_name in os.listdir(source_dir):
    source_file = os.path.join(source_dir, file_name)
    destination_file = os.path.join(destination_dir, file_name)
    
    # 既存のファイルを上書きする
    try:
        shutil.move(source_file, destination_file)
    except shutil.Error as e:
        print(f"エラー: {e}")
